In [1]:
import sys, platform
from pathlib import Path

# ----------------------------------------------------------------------------------- 
# NOTE:  Update the path below to point to the directory where `ub_camera.py` is saved.
#        `Path` make the / operator work on Windows, macOS, Linux
# ----------------------------------------------------------------------------------- 
CODE_PATH = Path("/home/cmurray3/Projects/IE-482-582/spring2026/Projects/code")
SSL_PATH  = CODE_PATH / "ssl"

import ub_camera
import ub_utils
import cv2

INFO: rospy is not installed and was not imported.  You may ignore this message.  Unless you are using ROS you do not need rospy.


In [2]:
# Start a camera instance
# Initialize `CameraUSB` Class
port = ub_utils.findOpenPort(8001, options=range(8000,8040))

paramDict = {'res_rows':480, 'res_cols':640, 'fps_target':30, 'outputPort': port}
apiPref   = None   # cv2.CAP_V4L2 # None
device    = 0 # 'https://10.83.17.66:8002/stream.mjpg'  # 0      # '/dev/video0' # 0          # '/dev/video0'


camera = ub_camera.CameraUSB(paramDict = paramDict, 
                             device = device, 
                             apiPref = apiPref,
                             sslPath = SSL_PATH,
                             showFPS=False)

camera.start(startStream=True, port=paramDict['outputPort'])

# Hide the frames-per-second text from the video feed
camera.showFPS = False

print(f'Visit https://localhost:{paramDict["outputPort"]}/stream.mjpg')

Logger:  No topic name/type found.  Just using print()
LOGGER: {'res_rows': 480, 'res_cols': 640, 'fps_target': 30, 'outputPort': 8001}
Visit https://localhost:8001/stream.mjpg
Camera Opened? True


In [6]:
# Show the frames-per-second text in the video feed
camera.showFPS = True

**Run the next line when you're ready to shut down the camera:**

In [16]:
camera.stop()

In [7]:
camera.keepStreaming

False

--- 

## Calibration
- See the `calibration_example.ipynb` notebook.
- NOTE:  You might want to calibrate the camera for `320x240`, too.

In [8]:
# This is copied from the camera calibration code.  See `calibration_example.ipynb`:
camera.intrinsics = { "640x480": {"fx": 613.9267755271052, "fy": 617.2876757419133, "cx": 326.06379688638367, "cy": 226.4726965669937, "dist": [-0.040671732389409375, 0.2205460570452358, -0.008313365917653356, 0.0025141234454979433, -0.32871689004906784]  } }
camera.intrinsics = camera._getIntrinsics()
camera.intrinsics

{'640x480': {'dist': array([-0.04067173,  0.22054606, -0.00831337,  0.00251412, -0.32871689]),
  'matrix': array([[613.92677553,   0.        , 326.06379689],
         [  0.        , 617.28767574, 226.47269657],
         [  0.        ,   0.        ,   1.        ]])}}

--- 

## ArUco Tags

In [3]:
# Specify the size of the ArUco tag in inches (or enter `None` if unknown)
TAG_SIZE_INCHES = None    #  4 + 3/16   

In [4]:
def aruco_post_poses(argsDict):
    # This function gets called each time an aruco detection is run
    idName  = argsDict['idName']

    if (TAG_SIZE_INCHES is not None):
        # Adjust based on resolution
        res = f'{camera.res_cols}x{camera.res_rows}'
        cameraMatrix = camera.intrinsics[res]['matrix']
        dist = camera.intrinsics[res]['dist']

        # ********************************************
        # Specify the size of the marker, in [meters]
        # ********************************************
        ml = ub_utils.inches2meters(TAG_SIZE_INCHES)  

        objPoints = np.array([[-ml/2,  ml/2, 0], 
                              [ ml/2,  ml/2, 0], 
                              [ ml/2, -ml/2, 0], 
                              [-ml/2, -ml/2, 0]])
        
    corners = camera.aruco[idName].deque[0]['corners']
    for i in range(len(corners)):
        # centers give the center point, in pixels, of the tag.
        print(f"id: {camera.aruco[idName].deque[0]['ids'][i]}")
        print(f"\tcenter: {camera.aruco[idName].deque[0]['centers'][i]}")

        if (TAG_SIZE_INCHES is not None):
            (ret, rvecs, tvecs) = ub_utils.arucoFindPose(objPoints, corners[i], cameraMatrix, dist, flags=cv2.SOLVEPNP_IPPE_SQUARE)
            # print(f"{ret=}, {rvecs=}, {tvecs=}.")
            if (ret):
                # rvecs is the 3D rotation vector.  
                # I don't think it's human interpretable, so we won't print it here. 

                # tvecs in the x/y/z translation of the marker from the origin (camera).
                # It's in [meters], since we specified `ml` in [meters].
                # tvecs[0] (x) is the distance left (-) or right (+) from the camera.
                # tvecs[1] (y) is the distance above (-) or below (+) the camera.
                # tvecs[2] (z) is the distance away from the camera.
                print(f"\tdistance [inches]: x: {ub_utils.meters2inches(tvecs[0])}, y: {ub_utils.meters2inches(tvecs[1])}, z: {ub_utils.meters2inches(tvecs[2])}")

In [5]:
# Specify what type of ArUco tag you have:
ARUCO_DICTIONARY = 'DICT_APRILTAG_36h11'

In [6]:
camera.addAruco(idName=ARUCO_DICTIONARY, 
                fps_target=5, 
                postFunction=aruco_post_poses, 
                postFunctionArgs={'idName': ARUCO_DICTIONARY}, 
                configOverrides={}, 
                ids_of_interest=None)  # default is None, or provide a list of IDs to track

LOGGER: Starting ArUco DICT_APRILTAG_36h11 thread at 5 fps
id: 10
	center: [309 435]
id: 11
	center: [462 330]
id: 10
	center: [361 323]
id: 9
	center: [466 205]
id: 11
	center: [432 354]
id: 9
	center: [430 224]
id: 8
	center: [329 218]
id: 10
	center: [324 350]
id: 11
	center: [418 347]
id: 10
	center: [312 342]
id: 9
	center: [418 217]
id: 8
	center: [318 211]
id: 11
	center: [407 347]
id: 10
	center: [304 342]
id: 9
	center: [409 218]
id: 8
	center: [310 213]
id: 11
	center: [394 347]
id: 9
	center: [397 219]
id: 10
	center: [290 342]
id: 8
	center: [298 214]
id: 11
	center: [373 371]
id: 10
	center: [270 366]
id: 9
	center: [377 241]
id: 8
	center: [278 236]
id: 11
	center: [382 373]
id: 10
	center: [280 366]
id: 9
	center: [386 246]
id: 8
	center: [290 240]
id: 11
	center: [397 371]
id: 9
	center: [399 247]
id: 10
	center: [295 364]
id: 8
	center: [305 243]
id: 10
	center: [314 370]
id: 9
	center: [415 254]
id: 8
	center: [322 250]
id: 11
	center: [414 376]
id: 11
	center: [437 3

**Run the next cell when you're ready to stop the ArUco detection:**

In [8]:
camera.aruco[ARUCO_DICTIONARY].stop()

LOGGER: Stopping ArUco DICT_APRILTAG_36h11 thread.


--- 

## Barcodes

In [9]:
def postBarcode(argsDict):
    # print(camera.barcode['default'].deque[0])
    for i in range(len(camera.barcode['default'].deque[0]['data'])): 
        print(f"""data: {camera.barcode['default'].deque[0]['data'][i]},  
                codeType: {camera.barcode['default'].deque[0]['codeTypes'][i]},
                quality: {camera.barcode['default'].deque[0]['qualities'][i]},
                corners: {camera.barcode['default'].deque[0]['corners'][i]}""")
  

In [10]:
camera.addBarcode(fps_target=5,
                  postFunction=postBarcode)

LOGGER: Starting barcode default thread at 5 fps
data: 0077890302774,  
                codeType: EAN13,
                quality: 5,
                corners: [(172, 249), (181, 366)]
data: 0077890302774,  
                codeType: EAN13,
                quality: 5,
                corners: [(152, 311), (178, 374)]
data: 0077890302774,  
                codeType: EAN13,
                quality: 6,
                corners: [(144, 313), (196, 376)]
data: 0077890302774,  
                codeType: EAN13,
                quality: 8,
                corners: [(123, 252), (186, 378)]
data: 0077890302774,  
                codeType: EAN13,
                quality: 8,
                corners: [(117, 252), (169, 377)]
data: 0077890302774,  
                codeType: EAN13,
                quality: 8,
                corners: [(128, 310), (167, 377)]
data: 0077890302774,  
                codeType: EAN13,
                quality: 5,
                corners: [(115, 252), (140, 373)]
data: 0077890

**Run the next cell when you're ready to stop the barcode detection:**

In [11]:
camera.barcode['default'].stop()

LOGGER: Stopping barcode default thread.
LOGGER: Stopping barcode default thread.
LOGGER: Stopping barcode default thread.
LOGGER: Stopping barcode default thread.


--- 
## Face Detection

In [12]:
# Need to indicate where the OpenCV face detection models are saved.
try:
    modelPath   = CODE_PATH / 'cv2_dnn_models'
except Exception as e:
    print(f'Error - Could not set face detection modelPath: {e}')

print(f'The OpenCV face detection models should be saved in the `{modelPath}` directory.')

The OpenCV face detection models should be saved in the `/home/cmurray3/Projects/IE-482-582/spring2026/Projects/code/cv2_dnn_models` directory.


In [13]:
def postFaceDetect(argsDict):
    # print(camera.facedetect['default'].deque[0])
    for i in range(len(camera.facedetect['default'].deque[0]['confidence'])): 
        print(f"{i} - confidence: {camera.facedetect['default'].deque[0]['confidence'][i]}, corners: {camera.facedetect['default'].deque[0]['corners'][i]}")


In [ ]:
camera.addFaceDetect(fps_target=5,
                     postFunction=postFaceDetect, 
                     conf_threshold=0.7, 
                     dnn='caffe',    # 'caffe' (fp16) or 'pb' (8bit)
                     device='cpu', 
                     modelPath=modelPath)

**Run the next cell when you're ready to stop the face detection:**

In [15]:
camera.facedetect['default'].stop()

LOGGER: Stopping FaceDetect default thread.
0 - confidence: 0.9989292025566101, corners: [(269, 70), (476, 387)]
LOGGER: Stopping FaceDetect default thread.
LOGGER: Stopping facedetect default thread.
LOGGER: Stopping FaceDetect default thread.


--- 
## Ultralytics
The following options are documented:
- Detect
- Pose
- Oriented Bounding Box (obb)
- Segment (mask)
- Track (can be applied to `Detect`, `Pose`, and `Segment`)

### Detect

In [3]:
def postUltralyticsDetect(argsDict):
    idName = argsDict['idName']
    results = argsDict['results']
    
    for result in results:
        '''
        xywh = result.boxes.xywh  # center-x, center-y, width, height
        xywhn = result.boxes.xywhn  # normalized
        xyxy = result.boxes.xyxy  # top-left-x, top-left-y, bottom-right-x, bottom-right-y
        xyxyn = result.boxes.xyxyn  # normalized
        names = [result.names[cls.item()] for cls in result.boxes.cls.int()]  # class name of each box
        confs = result.boxes.conf  # confidence score of each box    
        '''

        for i in range(0, len(result.boxes.cls)):
            # print(int(result.boxes.cls[i].item())
            # print(camera.ultralytics[idName].model.names[int(result.boxes.cls[i].item())])
            # print(result.boxes.conf[i].item(), result.boxes.xyxy[i].tolist())
            print(f'{result.names[int(result.boxes.cls[i].item())]} ({result.boxes.conf[i].item()}), {result.boxes.xyxy[i].tolist()}')

In [4]:
camera.addUltralytics(idName="detect", 
                      model_name="yolo11n.pt", 
                      conf_threshold=0.75, 
                      postFunction=postUltralyticsDetect)

LOGGER: Starting Ultralytics detect thread at 30 fps


/home/cmurray3/Projects/IE-482-582/spring2026/venvs/cam/lib/python3.12/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:119.)
  return torch._C._cuda_getDeviceCount() > 0


person (0.8884982466697693), [91.9151611328125, 106.83164978027344, 636.7255859375, 479.31396484375]
person (0.8830761313438416), [85.907470703125, 102.76670837402344, 623.37255859375, 479.916015625]
person (0.869709312915802), [87.95074462890625, 102.734375, 603.4698486328125, 479.30633544921875]
person (0.8585805892944336), [88.1065673828125, 102.36395263671875, 607.5125122070312, 479.9683837890625]
person (0.8532020449638367), [93.44671630859375, 101.96632385253906, 630.5538330078125, 479.34423828125]
person (0.8642722964286804), [92.6534423828125, 105.04597473144531, 629.8568115234375, 479.74444580078125]
person (0.8844556212425232), [95.64715576171875, 105.09078979492188, 597.1840209960938, 479.4951477050781]
person (0.8883975744247437), [95.68646240234375, 105.334716796875, 582.119140625, 479.529296875]
person (0.884955644607544), [97.75201416015625, 105.69004821777344, 600.0438232421875, 479.5257568359375]
person (0.8024802803993225), [101.43798828125, 103.8580322265625, 594.126

In [18]:
camera.ultralytics['detect'].model.names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [ ]:
camera.ultralytics['detect'].drawBox   = True
camera.ultralytics['detect'].drawLabel = True

In [19]:
camera.ultralytics['detect'].stop()

LOGGER: Stopping Ultralytics detect thread.


### Pose

In [4]:
def postUltralyticsPose(argsDict):
    idName = argsDict['idName']
    results = argsDict['results']
    
    '''
    `keypoints` should have 17 elements:
    0: Nose, 1: Left Eye, 2: Right Eye, 3: Left Ear, 4: Right Ear,
    5: Left Shoulder, 6: Right Shoulder, 7: Left Elbow, 8: Right Elbow, 9: Left Wrist, 10: Right Wrist,
    11: Left Hip, 12: Right Hip, 13: Left Knee, 14: Right Knee, 15: Left Ankle, 16: Right Ankle
    '''
    
    for result in results:
        if (result.keypoints.has_visible):
            print(f'conf: {result.keypoints.conf.tolist()}, keypoints: {result.keypoints.xy.tolist()} \n')


In [5]:
camera.addUltralytics(idName="pose", 
                      model_name="yolo11n-pose.pt", 
                      conf_threshold=0.75,  
                      postFunction=postUltralyticsPose, 
                      drawBox = False, drawLabel=True)

LOGGER: Starting Ultralytics pose thread at 30 fps


/home/cmurray3/Projects/IE-482-582/spring2026/venvs/cam/lib/python3.12/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:119.)
  return torch._C._cuda_getDeviceCount() > 0


conf: [[0.994666337966919, 0.994330644607544, 0.9971712231636047, 0.7679360508918762, 0.9599152207374573, 0.7971765995025635, 0.8693031668663025, 0.018396493047475815, 0.026801103726029396, 0.08032061904668808, 0.1119375228881836, 0.0012124330969527364, 0.0014310114784166217, 0.0015734710032120347, 0.0019066882086917758, 0.00198968593031168, 0.002229969948530197]], keypoints: [[[360.86395263671875, 341.42645263671875], [393.47943115234375, 294.84295654296875], [317.1319580078125, 298.785888671875], [431.28619384765625, 296.2500305175781], [243.69406127929688, 307.3488464355469], [519.5277709960938, 442.9888610839844], [127.44560241699219, 445.099609375], [585.8435668945312, 480.0], [57.969329833984375, 480.0], [460.7839660644531, 450.85589599609375], [210.23287963867188, 462.59698486328125], [447.1083068847656, 455.5777893066406], [207.79197692871094, 463.65325927734375], [478.5903625488281, 378.0149841308594], [236.67971801757812, 387.29620361328125], [381.1736755371094, 464.011108398

127.0.0.1 - - [27/Jan/2026 20:40:43] "GET /stream.mjpg HTTP/1.1" 200 -


conf: [[0.9944378137588501, 0.9965080618858337, 0.9959275126457214, 0.9311206936836243, 0.9179542064666748, 0.7844140529632568, 0.8785099983215332, 0.019183052703738213, 0.02808409370481968, 0.10510031878948212, 0.13253521919250488, 0.0011739748297259212, 0.0014520412078127265, 0.0017497786320745945, 0.0024358658120036125, 0.0021898699924349785, 0.0028999405913054943]], keypoints: [[[356.53704833984375, 325.3300476074219], [392.3133544921875, 280.13189697265625], [314.6771240234375, 284.175537109375], [437.17999267578125, 293.3299255371094], [251.6999053955078, 304.2395324707031], [523.513916015625, 449.9136657714844], [134.95953369140625, 450.4667053222656], [592.7677001953125, 480.0], [62.2174072265625, 480.0], [457.2410888671875, 465.467041015625], [237.57688903808594, 458.5751953125], [428.6561584472656, 462.16998291015625], [195.27197265625, 468.8056640625], [474.7249755859375, 387.20513916015625], [260.2499084472656, 394.55621337890625], [391.45977783203125, 480.0], [360.62008666

In [20]:
camera.ultralytics['pose'].drawBox   = False
camera.ultralytics['pose'].drawLabel = False

In [6]:
camera.ultralytics['pose'].stop()

LOGGER: Stopping Ultralytics pose thread.
conf: [[0.9930527806282043, 0.9960306286811829, 0.996027946472168, 0.9177587032318115, 0.9324505925178528, 0.6233522295951843, 0.8675580620765686, 0.001985829323530197, 0.00758508313447237, 0.011487608775496483, 0.030708985403180122, 0.0002340507780900225, 0.00046113153803162277, 0.0008354209712706506, 0.0015417411923408508, 0.002054488519206643, 0.003254513954743743]], keypoints: [[[344.0177307128906, 287.7262268066406], [383.47039794921875, 235.27685546875], [298.5533142089844, 235.60520935058594], [429.84161376953125, 249.568359375], [225.6245880126953, 252.51373291015625], [530.1763305664062, 455.43438720703125], [91.86907958984375, 440.933349609375], [542.3357543945312, 480.0], [34.995361328125, 480.0], [367.90966796875, 480.0], [195.92431640625, 471.24932861328125], [443.7142333984375, 466.47747802734375], [181.21170043945312, 466.97637939453125], [447.9202880859375, 380.98138427734375], [215.58670043945312, 383.21697998046875], [322.9243

### OBB

In [7]:
def postUltralyticsObb(argsDict):
    idName = argsDict['idName']
    results = argsDict['results']
    
    for result in results:
        if (result.obb):
            for i in range(0, len(result.obb.cls)):
                    print(f'{result.names[int(result.obb.cls[i].item())]} ({result.obb.conf[i].item()}), Center: {result.obb.xywhr[i][0:2].tolist()}')        

In [8]:
camera.addUltralytics(idName="obb", 
                      model_name="yolo11n-obb.pt", 
                      conf_threshold=0.65,  
                      postFunction=postUltralyticsObb, 
                      drawBox = True, drawLabel=True)

LOGGER: Starting Ultralytics obb thread at 30 fps


In [ ]:
camera.ultralytics['obb'].model.names

In [9]:
camera.ultralytics['obb'].stop()

LOGGER: Stopping Ultralytics obb thread.
LOGGER: Stopping Ultralytics obb thread.
LOGGER: Stopping ultralytics obb thread.
LOGGER: Stopping Ultralytics obb thread.


### Segment

In [10]:
def postUltralyticsSegment(argsDict):
    idName = argsDict['idName']
    results = argsDict['results']
    
    for result in results:
        for i in range(0, len(result.boxes.cls)):
            try:
                print(f'{result.names[int(result.boxes.cls[i].item())]} ({result.boxes.conf[i].item()}), {result.boxes.xyxy[i].tolist()}')   
            except Exception as e:
                print(f'Error: {e}')

In [11]:
camera.addUltralytics(idName="segment", 
                      model_name="yolo11n-seg.pt", 
                      conf_threshold=0.65,  
                      postFunction=postUltralyticsSegment, 
                      drawBox = False, drawLabel=True, 
                      maskOutline = False)

LOGGER: Starting Ultralytics segment thread at 30 fps
person (0.8643827438354492), [20.4884033203125, 36.08793640136719, 627.2822265625, 478.80718994140625]
person (0.863937258720398), [20.83660888671875, 37.13275146484375, 627.769287109375, 478.75347900390625]
person (0.8599948287010193), [18.307373046875, 36.93121337890625, 626.97802734375, 478.76885986328125]
person (0.8859093189239502), [18.60693359375, 37.0736083984375, 626.3409423828125, 478.6947021484375]
person (0.8863078951835632), [18.8436279296875, 36.95848083496094, 625.0610961914062, 478.7960205078125]
person (0.8736376166343689), [20.79620361328125, 36.59130859375, 627.4155883789062, 478.65106201171875]
person (0.8826298713684082), [19.87091064453125, 36.27870178222656, 625.9439697265625, 478.76513671875]
person (0.8754737973213196), [19.95281982421875, 36.013916015625, 625.8899536132812, 478.7730712890625]
person (0.8658747673034668), [19.5467529296875, 36.4576416015625, 626.1932373046875, 478.744873046875]
person (0.861

In [ ]:
camera.ultralytics['segment'].maskOutline = True

In [28]:
camera.ultralytics['segment'].model.names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [12]:
camera.ultralytics['segment'].stop()

LOGGER: Stopping Ultralytics segment thread.
person (0.8879631161689758), [26.92431640625, 45.724639892578125, 627.8280029296875, 478.7638244628906]
LOGGER: Stopping Ultralytics segment thread.
LOGGER: Stopping ultralytics segment thread.
LOGGER: Stopping Ultralytics segment thread.


### Track

In [13]:
def postUltralyticsTrack(argsDict):
    idName = argsDict['idName']
    results = argsDict['results']
    
    # print(idName)   # "track"
    for result in results:
        '''
        xywh = result.boxes.xywh  # center-x, center-y, width, height
        xywhn = result.boxes.xywhn  # normalized
        xyxy = result.boxes.xyxy  # top-left-x, top-left-y, bottom-right-x, bottom-right-y
        xyxyn = result.boxes.xyxyn  # normalized
        names = [result.names[cls.item()] for cls in result.boxes.cls.int()]  # class name of each box
        confs = result.boxes.conf  # confidence score of each box    
        '''
        for i in range(0, len(result.boxes.cls)):
            try:
                print(f'ID: {result.boxes.id[i].item()} - {result.names[int(result.boxes.cls[i].item())]} ({result.boxes.conf[i].item()}), {result.boxes.xyxy[i].tolist()}')                
            except Exception as e:
                print(f'Error: {e}')

In [ ]:
# Tracking can be done with detect, pose, or segment models:
model_name = "yolo11n.pt"       # detect
# model_name = "yolo11n-pose.pt"
# model_name = "yolo11n-seg.pt"

camera.addUltralytics(idName="track", 
                      model_name=model_name, 
                      conf_threshold=0.65,  
                      postFunction=postUltralyticsTrack, 
                      drawBox = False, drawLabel=True)

In [5]:
camera.ultralytics['track'].drawBox = False
camera.ultralytics['track'].drawLabel = True

In [ ]:
camera.ultralytics['track'].stop()

In [13]:
camera.stopStream()
camera.stop()

--- 
## ROI
- Probably won't document this function anytime soon

## Timelapse
- TBD

## Video from Pics
- TBD